In [1]:
#default_exp core

In [2]:
# !pip install nbdev

# restaurants_timeseries

> API details.

In [3]:
#hide
from nbdev.showdoc import *

In [4]:
import os
import pandas as pd
DATAPATH = 'data'

DATA_FILES = {
    "visits": os.path.join(DATAPATH, "air_visit_data.csv"),
    "store_info": os.path.join(DATAPATH, "air_store_info.csv"),
    "reservations": os.path.join(DATAPATH, "air_reserve.csv"),
    "date_info": os.path.join(DATAPATH, "date_info.csv")
}

def read_data():
    return dict([(name, pd.read_csv(path)) for name, path in DATA_FILES.items()])

#export
data = read_data()

In [5]:
for name, df in data.items():
    print(name)
    display(df.head())

visits


,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


store_info


,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


reservations


,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


date_info


,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [6]:
from nbdev.export import *
notebook2script()

Converted 01_view_data.ipynb.
Converted f00_core.ipynb.
Converted index.ipynb.
